In [1]:
import keras
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
from tensorflow.keras.layers import Conv2D, Input, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model

In [14]:
path = './train/cup/0003.jpg'
image = load_img(path, target_size= (299,299))
arr = np.array(image)
x = np.array([arr])
x = preprocess_input(x)

model = Xception(weights= 'imagenet', input_shape=(299, 299,3))
pred = model.predict(x)
output = decode_predictions(pred)

In [4]:
df_train_full = pd.read_csv('/content/drive/MyDrive/kitchenware-classification/train.csv', dtype={'Id': str})
df_train_full['filename'] = '/content/drive/MyDrive/kitchenware-classification/images/' + df_train_full['Id'] + '.jpg'

val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   horizontal_flip=True, rotation_range = 90, zoom_range = 0.2)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(240, 240),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(240, 240),
    batch_size=32,
)

Found 4120 images belonging to 6 classes.
Found 1439 images belonging to 6 classes.


In [29]:
basemodel = Xception(weights= 'imagenet', input_shape=(299, 299,3), include_top= False)

for layer in basemodel.layers:
    layer.trainable = False

input = Input(shape = (299, 299, 3))
base = basemodel(input, training = False)
conv_lyr = Conv2D(64, (3,3), activation = 'relu')(base)
conv_lyr = Conv2D(64, (3,3), activation = 'relu')(conv_lyr)
pooling = MaxPooling2D(pool_size= (2,2))(conv_lyr)
dense = Dense(128, activation = 'relu')(pooling)
flatten = Flatten()(dense)
dense_2 = Dense(6)(flatten)
model = Model(input, dense_2)

lr = 0.01
optimize = keras.optimizers.Adam(learning_rate=lr)
loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimize, loss = loss, metrics= ['accuracy'])
history = model.fit(train_generator, epochs=10, validation_data = val_generator)

In [4]:
model = keras.models.load_model("C:/Users/Godwin/Downloads/xception_v4_07_0.960.h5")

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/kitchenware-classification/test.csv',
                      dtype={'Id': str})
df_test['filename'] = '/content/drive/MyDrive/kitchenware-classification/images/' + df_test['Id'] + '.jpg'

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(240, 240),
    batch_size=32,
    shuffle=False
)

model = keras.models.load_model('/content/xception_v5_09_0.944.h5')
y_pred = model.predict(test_generator)
classes = np.array(list(train_generator.class_indices.keys()))
predictions = classes[y_pred.argmax(axis=1)]

df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('/content/drive/MyDrive/kitchenware-classification/images/'):-4]
del df_submission['filename']

df_submission[['Id', 'label']].to_csv('colabsubmission12.csv', index=False)

In [10]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite = converter.convert()

with open('lite_model.tflite', 'wb') as f:
    f.write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Godwin\AppData\Local\Temp\tmpwbl9qjv0\assets


INFO:tensorflow:Assets written to: C:\Users\Godwin\AppData\Local\Temp\tmpwbl9qjv0\assets


In [4]:
from PIL import Image
import numpy as np
path = './images/0004.jpg'
with Image.open(path) as f:
    img = f.resize((299, 299), Image.NEAREST)

def preprocess_input(x):
    x = np.array(x, dtype = 'float32')
    x /= 127.5
    x -= 1.
    return x

arr = np.array(img)
x = np.array([arr])
x = preprocess_input(x)

C:\Users\Godwin\AppData\Local\Temp\ipykernel_13324\4002747960.py:5: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = f.resize((299, 299), Image.NEAREST)


In [5]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path = './lite_model.tflite')
interpreter.allocate_tensors()


input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, x)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

classes = ['cup','fork', 'glass', 'knife', 'plate', 'spoon']
out = dict(zip(classes, preds[0]))
out

{'cup': -0.5970967,
 'fork': -3.2493312,
 'glass': 0.76704663,
 'knife': -2.1284857,
 'plate': 6.322213,
 'spoon': -1.6190956}

In [10]:
from keras_image_helper import create_preprocessor
preprocess = create_preprocessor('xception', target_size=(299,299))

url ='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAoHCBcWFRgVFhYZGBgaGhoYHBoaH\
    BwaGh4aGhoaGhoaGhoeIS4lIR4rIRgaJjgmKy8xNTU1GiQ7QDs0Py40NTEBDAwMEA8QHhISHzQrJSs0NDQ0NDQ\
    0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NDQ0NP/AABEIALYBFQMBIgACEQEDEQH\
    /xAAbAAACAwEBAQAAAAAAAAAAAAACAwABBAUGB//EADwQAAEDAQUGBAQFAwQCAwAAAAEAAhEDBBIhMWEFQVFxg\
    ZETIqGxMsHR8AZCUmLhcoLxFJKislNjIzND/8QAGgEAAwEBAQEAAAAAAAAAAAAAAAECBAMFBv/EACcRAAIC\
    AgIBAwQDAQAAAAAAAAABAhEDIQQxEhMioTJBUWGRsfCB/9oADAMBAAIRAxEAPwBzakb1DXQERyTbNZi9\
    4a0ST2A4nRealekes2ltjmVjktDHlek2b+GKcAvJec8MB03rqj8P0B+T1d9V0XHbOEuRFPR4umeqYxnAL1lT8OU\
        TlebyM+8rHW/DzhJa4O0Ig9xKHx5IS5EWcZlHRMDAjfTezBzSBxzHcKg8fcrnKDjpotTUtphsA4I3DolNqfcJrX\
        aqSgboULJxTEJKQrF3VV1NlQoHYsA9ERIV3kLkIC5RApYV3/uCiwoY0KyOCUDzRtdKdioqToraJRXcFXhooLKIVEq\
        zTKFxO/JAyi0JTmQjcN4UalYCHFLeSm1As75G6eSC0AXkFG1/FLqaIQ0qSqH4pzKkDH6LO0pjDJQiWamnVQtQTCMHVd\
        EQwYUQOOqiYzyvjDjou1sesGNa4/mkzpJA9p6ryDrpMiCNxmQvS7MqDw6Y/YB2lp9QV34yTk3+jhynUVX5Pf7OtwIA3rq\
        CqF4KzVS2BPr7Hh98u1ZtoEjOVq8fwYnJHpG1QTCMlcejaMQV1mGRKtqibs8rt1j2OkExuIXnX7RDTD8D+oZdvovo9pptcIcJ\
        C8jtz8NhzSWSRw3jktEPTyR8ciMk/VxS8sbMdGuCAQQeS0sevG1Kj7K/HEE5cV6LZu0GVG32nDIg5g8D6471h5fAli90Nr+j\
        fxOdHN7Zqn/Z2mlXCQwz9hFejf7LzzeW5qXe4qzVnInsT2wUaZyntCCggdPZURorA5qQHce7gEAU4JRfCaW6RyUuoBAF+qsuGvYohT\
        O8I20iigtAtmURe2fiAPCR7JjGQjkjdKqhWUCM5wQPb1R9FJQIzvYRwQEBa3HDd7LPVplJoExD2HclFq0oXBKirMVycYjdjHyVBsYHFa\
        Htn5Km09UqLsSExk7o6oiwIG6JJUHY5zwMDmoYHASozPJNIXREMWop4ep9PoogD57Xpkbz6fROsduLGAfoeR0fLh630dsEa/Nctjxf\
        LDF14uycgZ8pPJwHQldePLxnvonkY3PG67Wz19i2kHAD7C7FmfkRiF8ysttcxxa6RBII4EYQva7G2jIE5L1HGjyU7PZWd8rt2av5V5azW\
            ldGzWoDMpKNilLxO258pT3LI20A71ZrJqJylM89+Ldjiq2+0C8MSOMcNV5f8E7Fc+u/zENa2DqCcuc+0r6JUdKCwUWUyS1oaXYkjCTr3Wj1H6bgZ4pLJ5I4VVrqTyx+7EHcRuK1Stf4noh1MVB8VMz/AGn4h8+i5FmfIwXh54eEv0z6DBP1I3912bpUJCTeRNes9nWgi5La469U3FVMZpjCZjuT2tVUjhKu9jAVIhssuAzUvjgexSizGTM8iFbg0YzHX5IChjnb4KX4g4P7FC7mfRRpnf7fJMdDbo1Sy37lC+mZGJw/cR6AYq2ka9f8pAQMxm6NckyJQtfPHsVb3oQCS0IHhWSEDsCkApzsMlCie2O6oDGfmUUVYp1PmiunqjkJzGb0qHZTG4c0QGvoju8lCIVk2CQoiIUQI8tbbIDK8vtKwxOC+jVbGCuRb9k3gVyVp2dVJHgLZ5meJmWw1/LJtTrkdY4puztp3SAD96rp2rZj2OvAAiCCCMHNObXaELzG0bGaTg5sljvhJzB/Q79w9c17PFzxyR8X2jzOVg8ZeUen8M+k7M2iHAYrt2avK+VbL2qBAM7l7nZW0WuGc6rRKNdGVfs9fSIKN7SB8RCxWWqCEVavOC5q2xS8Ui77yYBPZPbRfMB554BZqT4x4cVpoV98pybFGKNAsRghzyQRBBJOHVeesIIDmHNpLT0JHyXbqbRY0+Y9sSuPStDX1XlvwuhwB1EH1BWLlRcop/g3cSSi2vyaIRtCIUtETacLAkbWwQDxTA1QNTAFYi4gRKNoKqMEwBBFglLc1PhA4JMaFg6KpVkKkyhTwf1ERjhHrIVOdr2+4TXBJLYEnAYQPkkBKLpEwesT6YIXuxT4WdxxQBL4QAnLcrdiiDMEFCxiiawlGxi0MZCKJbF06OgTHNAzGHonBsoAwCc+pJ7J0TYi4ZmcNwA+5TGkHFMMckM44Dqih2S6qQkkcSoi0GzPTpvnFW+g5b7iosUKIeRxbTYARiF5jamx8CLoc05tO/ruI4he9dRnefb2WWtYQ4Ru1n6pryi1KPYWmqfR8T2ls51E3hJZOBOY0d9citOytrXDjP37L6Vb9gNIMAegXjtofg94Jcxt3li30xHSeS9XBzIy9uTT+DFl4zXuhtfJ3tl7cBaIM6LrNt7Sc8dV81e20UJApwB+ceZo5kZcjont/EFxnm8zzmRu7rZ4rtGJp9M+iV9otAicVnftgEXWZ+q8ZZNp+JiBJ1AXZZZXxMXJ3vMDoDn0USSitgk5PRe07Y6brTjvOMAfVcZ34gfZ3sc3zAfE1xzBiYO5dqnsuZLnk8mP9DdC59XYFBzjeqG9vvseOhgGFweTG/a2jTHHOO0mez2H+IqNpaCx3mGbCYcObd/PJdsPBXzezfg4vIdRg3fzMeA4a53m+i7NCtbaGDmGuwYcKgjUYO6jqsk+N94O/wBGmOddSVHr4V3QuRs3b1KqbgJa/fTeLr8M8D8XNpIXTBWZxcXTOykpK0PYQmys7StIxyQhMB6AuTHtSi3H790hoAGcsUV1XcicULgeHPP79UFIhYp4fHFGxuGSFz44HklQWLrOgQs6a4IQ3FDKQLAtFHEEIWNTGsQiZEayEbQMvniqaN/3/lMjBUSU7TFQjeeu9WGxvQPfCYig0EYZHSFHEBKqWkJBqE6e6lstRY11Yc1Fkc77CimyqOy1QBK8VUKw4p2cqY4hLLVXiDiq8ccUWFMjmJNSyg7k3xhr2KgqjgmPZgfslrjN3Eb96U7YLDm29zDXf9gV1W1dR2/lMbWTUmunQNX2jmWbYrRuLeRgdmrZT2XTbjdjkSPYrWKiIPTbvsXXQk0mgfD99Uh9ka4YiNIH1W0jggIdwB9Egs87bthMzabjtzm4EHnxS7NarRSHnu1Wanz/AO8DHqF6dw3JT7OHDED3VRco/SxSqSqSs4b6NltQh7W3+DxDwRwOnEJA2faKJhj/ABWbmVCb0ftqDP8AuB5rdbNjtdjBkZaHRZ6VpqUsHgvZrF8fJ3vqtUc0ZLxyIzywyi/LG/8AgVk2mx7rjg6m/wD8b/K7+3GHDVpK6TDiMSkVKVK0shwa4DfHwndeHxMOvqsjLHWo4Md4jNzKh8wH7am/+6eamXG+8GOPI3UlR2xUULxwXPstva83cWP3sf5Xdt41Ehb2tWWUXF0zQmmrQJqhQvRmlKgpapbK0Ic8oHCVo8LVUWpMdiWU05rEYVppCbF3eSMoy1Ux/f73KkS2WGxnioVHuWZ75wSbBKy31FmqOOn30RuVZ8eils6JULJ7qrhO9aW0xmERaih2Z/D6qJ0KIoVmQOPDur83ALXgrJCmh2ZGU8cmz/T/ACjFUZY9jHdPvt09FV9vPlinQrFXlbXJrSDu90VwcEUxWB4ikptwKBg4J0FoXehQPP3/AITDRCE0Ac8fVFMLRbKsf5Tm1Zz+izmjwR3HTujh9lCsTSH31fiLM5hyI7FD4YGN3vmnbFSNJIP+UirRDlUDghPVFh4nOrbOLXX2OuuGRbIPUzBGiqntWDcrtLP/AGNEsn9zBi06tw4wum52G9YqrQ8GWxuxC6wyyh0TLFGa2PtdkZUYC8NcDi14Mg8Cx436rC4WiiTcPjsAm4SBVaP2uPlfydB/cue/xbOS6jBacXU3YsdzbuOogro7M23TqkNHkqj/APN5xnf4b940K1xyY8y8ZdmSWLJh90do22DabKktaSHt+Jjhde3+pjsYwwOR3Sug12nsuXb9nMqQSMW4jMOad9x4hzDyKy+PWpZnxG/uhtQcMcGP/wCJ/qK5T47juOzpjzp/UeiuhC5usdlyrNtdr8AYcM2mWuHNpEjqtX+pnesz1qjQk3s0AEbye3yCtvM+n0WXxio554lTY/E1OclPqRkk48CVQaeQQ2NRKe88J9+ijaR5SmhoHNVHMoHZTWQqlWGk7u6nhpUFlSgLzOWHr2hNLIQ3xlIlMALx/wASPmomQogBCpzshE9sOcoWVQdx64K/Gb9kKLKoJsK+qX4w3D0J9kTKn3dI907Cg2uR3kt71Ximcj6fVMRoDuaKCkio7TujDjxCLFQYBVgJUOO/0P1TAw8UxBGdwlWwnhCJrdVZbqmKwTjKC7qUy6qLECALVRMZ+6aAruoodmcgnJwjv81VQRnlx3dVphVIQFmQ2YHgVydo7AZUGLQDuIzC9Ar6JUNNo8nTt1ezQKgNWmML4+No4H9Q0Pou9ZrVTtDJa5rgcOvAjMHT3TqtIOwIELhW3YrmO8SzuuP0EtOjhMEdFpx8iUdS2vk4ZMEZbjp/A23bJIILcQMgZkf0Ob5m9MNIS6FsewgOl4yg3Wv6H4H/APE6FN2d+IQXeFaG+G/ISfI/+lxy5HvkF17TZGvBwzz/AJBEHqFqcYZVfZlUp4nTE2a2sfIBMjNpEOHNpxC0rlWjZs8fLkWyY5Cb7f7SQpZ31WcKreYv94APUN5lZp8dr6TTDOpdnXuqXFns9uY83QSHZlpF1w6HdrktQcszjTpndO9ooNVFpV+IFRqBIAwoUAqbpHfFDjMmI7/JMQT4zQhyFxxyxQuekVQQ5k9vkFEF9RA6MsfcqMaBvPVxPuVnZV0TGVea5JotpmoBWkCqi8Uce6q0KmMqNnfHKPmELKepP3oltrt/UO6LxdU7QtjA37ko2lINTVA+1ADNFodM3ByK+OKwMtIOXXAhNFTgnaE4mnxxx9VT6gOE9jCV4isPRYqGmeXVQT+odv5S7yFryTFw88PrKdioZWvbj2AJ9SFGuP3Ck6KE6IAptQzEGOOH1lGHalCHI0AU2pPHqI91V5USqvFKwoniaIHVT+k/8fqivlVeRZVGK2WNlZpa9hg8bvpBXLpVLRZMMa1Ebs6jG6H8wHD6yvRToo4TuVxnKDuLJlFSVSWhNi2gys2+xwOOO4g8CM2n7xRvYCcQZ/U2A8cxk4fcFcm27Fl3iUnGnUH5hk7R7cnBFZ9puBDLQ0U37nA//G/+lx+E6Fb8WeMtS0zBl40oe6O18m202Vzhi0VGjIj4geMZtdyxWRtV7MGukfoqT6PiR/cDzC6LKpB5b8j3THvY8Q9oOuAcPvTsu0sUZaaOMc0ovTMtPaDMA8FjjkHRBP7XAlpPIrY3cRI0w+Sqy7PYCYh7HYFjsZHIqq2wwPNQe5n7T56fK6TI6ELHPjpP2s1wzt/Ui3P1HZQvK5htT2SHsvAZup+fuyL47Hmn2a2Mf8DweR38DwKzyhKPaNMZRl0zSXEoXyoAZ4q7qgsAg7j7qIo1URQWZLqaWq7rv28pPvHyVsB3gd5+S5UOwLg0ULEbr25oPX+FY5J0wsS5gyV3RwTp0SnVAPyu7E+yEgslwcETWDgoysP0OPSPcqyTmGd3R9Uxl3dExoVMDoxaB1n1hNY074HLH5BAmVdVhqs0ycnEdlbKZ3k+iYrKLN6gBVOs0/nf0IHsEbKMbyeZlGxWVBURmnulU2jG89ST7lUKyuiq8p4Jn43Rw8sf9Z9Vdxw3g+iA0C5CQmvadwHUpV136W6+Y+nlSGgSrBRXdJ5fyrAP6R3xRQ7KbCuQrDUwNQKxRhLr0WPaWvaHNO4iVqAV3U6FZwv9HUo//XNSn/43HztH/rcc/wCkrTZ7Qx4N0zGYODmng5uYXSfTO4kco+YWO07MY8h8uDxlUEBw0MASNCtOLkSjqW0Z8uCM99MawgiN4xH8aq6VuewxM8JxnqMe8rmVbW6iQK4gYAVW/AT+8fkPpyTKpvCZ1BzB4EFboyjNWujFKEsf+7Oy+zsqQ8CHb44hZbRsim83nDzbng3Xjk8QVnsVrLXSf6SF0rQ+6ZGWBQ41oFK9o5dSw16eLH+KwZtIAqAftPwu5EDmtFFjXC9iQ7jPscuS6VmtEhcupUAe4DjO7MrJnxxS8kjXgySk/FseKTW4Q0cgFEPi6H0+qiy2jTTM19WH6JUo7y5JnShgerDwckq8qdBzAPNFhRpCsALMwxhACK+nYUOJHBEHLM5w4AqeI0ZDsD8giwo0X0TXjNZPGGvYq/Hbx+SLF4my+FZeBiT6rG2qOI7hN7J2FDW1mnJwPUIi9IJQOxzHqixUPNREHrK1oH+SfmjEJWOjUHqeIsheBx7E+wRsKdi8R5qIS/RLUlFhQxlYHBMkLOCo5gzjHjCLCh4Vhyzjr3/lXj9kIsVD5UDxxWcz9uj5KEHTuSiwoeShDscwlYbwD0RTHDsmFBvAIIIBBzBxB5ri19mPpy6zwW5mi74DxuH8p9F2L6E1RxVQnKDtMUoKSpo4dntTHksHkfmab/K7m05OHJdZlU3A1+Y8pgzIO/2Wa32KnVHnbJGRxkcjuXPdY6zcGV3FoyDmtcf9xE91sjy01UjM+JTuP8M61ltBaCTgBmTkAk2Z19znnI5ct33qsbbO9xBe8vjjAA6DBbGmAuObN56XR1w4PT23bNd1RZL7uKiznehf+oncp4qii4o6UH4kohUUUT+5JTamiIVFFE0BC9WIPHuVFEAUQMse5+qHA8e5+qiiYDWlHfwVqJiZV5QvUUQxgGolufGQ+SiiT6Eg/ElTxSFFEIBjaiIFRRMGS+VPFKiiYiGoVQrFRRA0AaxVCoVFEhhOepKpRMRccFfhaqKJgRtDVW2krUQIs2cIf9OM5PLD6K1EAGykGiAooomI/9k='
x = preprocess.from_url(url)

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, x)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

classes = ['cup','fork', 'glass', 'knife', 'plate', 'spoon']
out = dict(zip(classes, preds[0]))
out

{'cup': -1.3186173,
 'fork': -0.41499886,
 'glass': -0.43558893,
 'knife': -3.4201918,
 'plate': -3.1137578,
 'spoon': 8.817198}